<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/CAFE_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:

from huggingface_hub import notebook_login

notebook_login()

In [3]:

from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:

import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
%%capture
!gdown --id 1EZTvcHo2x3cim4uSTkfMHoagipfQtLak
time.sleep(5)

!unzip /content/FESC_CaFE.zip

In [7]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/CaFE/CaFE_splits/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/CaFE/CaFE_splits/test.json",'r') as f:
    test_data = json.load(f)

In [8]:
train_data

{'09-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-4.wav',
  'length': 3.5731875,
  'emo': 'happiness'},
 '09-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-5.wav',
  'length': 3.8738125,
  'emo': 'happiness'},
 '04-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/04-J-1-6.wav',
  'length': 5.84675,
  'emo': 'happiness'},
 '02-J-1-1': {'wav': 'CaFE_48k/Happiness/Faible/02-J-1-1.wav',
  'length': 4.8348125,
  'emo': 'happiness'},
 '10-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/10-J-1-6.wav',
  'length': 4.38675,
  'emo': 'happiness'},
 '10-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/10-J-1-4.wav',
  'length': 4.6013125,
  'emo': 'happiness'},
 '02-J-1-3': {'wav': 'CaFE_48k/Happiness/Faible/02-J-1-3.wav',
  'length': 4.6151875,
  'emo': 'happiness'},
 '04-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/04-J-1-4.wav',
  'length': 4.626,
  'emo': 'happiness'},
 '09-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-6.wav',
  'length': 3.7488125,
  'emo': 'happiness'},
 '04-J-1-5': {'wav': 'CaFE_

In [9]:
test_data

{'11-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-6.wav',
  'length': 5.1439375,
  'emo': 'happiness'},
 '11-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-4.wav',
  'length': 4.9079375,
  'emo': 'happiness'},
 '11-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-5.wav',
  'length': 4.6163125,
  'emo': 'happiness'},
 '11-J-1-1': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-1.wav',
  'length': 4.2954375,
  'emo': 'happiness'},
 '11-J-1-2': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-2.wav',
  'length': 4.0303125,
  'emo': 'happiness'},
 '11-J-1-3': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-3.wav',
  'length': 4.2556875,
  'emo': 'happiness'},
 '12-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-6.wav',
  'length': 5.267,
  'emo': 'happiness'},
 '12-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-5.wav',
  'length': 5.34375,
  'emo': 'happiness'},
 '12-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-4.wav',
  'length': 5.4876875,
  'emo': 'happiness'},
 '12-J-1-1': {'wav': 'CaF

In [10]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)


In [11]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [12]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
})

In [13]:
def add_base_path(example):
    example['wav'] = '/content/CaFE/' + example['wav']  # Adjust column name if it's not 'wav'
    return example

dataset = dataset.map(add_base_path, batched=False)
dataset['train'][0]

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

{'id': '09-J-1-4',
 'wav': '/content/CaFE/CaFE_48k/Happiness/Faible/09-J-1-4.wav',
 'length': 3.5731875,
 'emo': 'happiness'}

In [14]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [15]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
})

In [16]:
dataset['train'][0].keys()

dict_keys(['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'])

In [17]:
%%capture
!git clone https://github.com/aalto-speech/mtkd4ser.git

In [45]:
script = """
import torch
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import torch.nn.functional as F
import warnings
import json
import argparse
from transformers import Wav2Vec2FeatureExtractor, AutoModelForAudioClassification
from datasets import DatasetDict, Dataset
from torch.utils.data import DataLoader
import random
import numpy as np

warnings.filterwarnings('ignore')

# Emotion filtering for CaFE-specific emotions
def is_common(emotion):
    return 1 if emotion in ["neutral", "sadness", "happiness", "anger"] else 0

# Preprocess audio using Wav2Vec2 feature extractor
def preprocess_function(examples, feature_extractor):
    max_duration = 20.0  # seconds
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        padding=True,
        truncation=True
    )
    return inputs

# Collate function to handle variable-length inputs
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: len(x["input_values"]), reverse=True)
    inputs = [torch.tensor(example["input_values"]) for example in batch]
    labels = [int(example["label"]) for example in batch]
    inputs = pad_sequence(inputs, batch_first=True)
    return {"input_values": inputs, "label": torch.tensor(labels)}

# Training function
def train(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_predictions = 0
    correct_predictions = 0
    total_train_loss = 0
    all_actual_labels = []
    all_predictions = []

    for batch in train_loader:
        inputs, labels = batch['input_values'].to(device), batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs).logits
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        total_train_loss += loss.item()
        all_actual_labels.extend(labels.tolist())
        all_predictions.extend(predicted.tolist())

    total_train_loss /= len(train_loader)
    train_accuracy = correct_predictions / total_predictions
    unweighted_recall = recall_score(all_actual_labels, all_predictions, average='macro')
    weighted_recall = recall_score(all_actual_labels, all_predictions, average='weighted')
    return unweighted_recall, weighted_recall, train_accuracy, total_train_loss

# Model setup for fine-tuning
def model_ft(label2id, id2label, num_classes=4, device="cpu"):
    MODEL_CKPT = "facebook/wav2vec2-base"
    model = AutoModelForAudioClassification.from_pretrained(MODEL_CKPT, num_labels=num_classes, label2id=label2id, id2label=id2label)
    model.freeze_feature_encoder()
    model.to(device)
    return model

# Save the model and feature extractor
def save_model_and_extractor(model, feature_extractor, model_save_path, feature_extractor_save_path):
    torch.save(model.state_dict(), model_save_path)
    feature_extractor.save_pretrained(feature_extractor_save_path)
    print(f"Model and Feature Extractor saved to {model_save_path} and {feature_extractor_save_path}")

# Main function
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--N_EPOCHS", type=int, default=10)
    parser.add_argument("--LEARNING_RATE", type=float, default=1e-5, choices=[1e-5, 3e-5, 5e-5])
    parser.add_argument("--BATCH_SIZE", type=int, default=8)
    parser.add_argument("--MODEL_SAVE_PATH", type=str, default="trained_model.pth")
    parser.add_argument("--FEATURE_EXTRACTOR_SAVE_PATH", type=str, default="feature_extractor")
    args = parser.parse_args()


    # Now you can access these arguments using `args.SESSION` and `args.TRAINING`
    print(f"Session: {args.SESSION}")
    print(f"Training: {args.TRAINING}")
    random_seed = 42
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(random_seed)

    # Argument parsing
    N_EPOCHS = args.N_EPOCHS
    LEARNING_RATE = args.LEARNING_RATE
    BATCH_SIZE = args.BATCH_SIZE

    # Set paths for the train and test data
    train_path = '/content/CaFE/CaFE_splits/train.json'
    test_path = '/content/cafe/cafesplits/test.json'

    # Now loading the data from these paths
    with open(train_path, "r") as f:
        train_data = json.load(f)
    with open(test_path, "r") as f:
        test_data = json.load(f)

    # Prepare dataset
    label2id = {"neutral": 0, "sadness": 1, "happiness": 2, "anger": 3}
    id2label = {v: k for k, v in label2id.items()}

    # Convert data to DatasetDict format (for compatibility with Hugging Face's datasets library)
    train_dataset = Dataset.from_dict(train_data)
    test_dataset = Dataset.from_dict(test_data)
    ds = DatasetDict({"train": train_dataset, "test": test_dataset})

    # Initialize feature extractor
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")

    # Apply preprocessing function to encode the audio
    encoded_audio = ds.map(lambda example: preprocess_function(example, feature_extractor), remove_columns="audio", batched=True)

    # DataLoader setup
    train_loader = DataLoader(encoded_audio["train"], batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(encoded_audio["test"], batch_size=BATCH_SIZE, collate_fn=collate_fn)

    # Model and optimizer setup
    model = model_ft(label2id, id2label, len(label2id), device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(N_EPOCHS):
        unweighted_recall, weighted_recall, train_accuracy, train_loss = train(model, train_loader, optimizer, loss_fn, device="cuda" if torch.cuda.is_available() else "cpu")
        print(f"Epoch {epoch + 1}/{N_EPOCHS} - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Recall (macro): {unweighted_recall:.4f}, Recall (weighted): {weighted_recall:.4f}")

    # Save the trained model and feature extractor
    save_model_and_extractor(model, feature_extractor, args.MODEL_SAVE_PATH, args.FEATURE_EXTRACTOR_SAVE_PATH)



 # Print the arguments to verify they're being parsed correctly
    print(f"Session: {args.SESSION}")
    print(f"Training: {args.TRAINING}")
    print(f"Epochs: {args.N_EPOCHS}")
    print(f"Learning Rate: {args.LEARNING_RATE}")
    print(f"Batch Size: {args.BATCH_SIZE}")
    print(f"Model Save Path: {args.MODEL_SAVE_PATH}")
    print(f"Feature Extractor Save Path: {args.FEATURE_EXTRACTOR_SAVE_PATH}")


if __name__ == "__main__":
    main()

    """

# Save the script content to a .py file
with open('/content/test_args.py', 'w') as f:
    f.write(script)


In [46]:
!python test_args.py --N_EPOCHS 10 --LEARNING_RATE 1e-5 --BATCH_SIZE 8 --MODEL_SAVE_PATH /content/trained_model.pth --FEATURE_EXTRACTOR_SAVE_PATH /content/feature_extractor --SESSION 1 --TRAINING 1



usage: test_args.py [-h] [--N_EPOCHS N_EPOCHS] [--LEARNING_RATE {1e-05,3e-05,5e-05}]
                    [--BATCH_SIZE BATCH_SIZE] [--MODEL_SAVE_PATH MODEL_SAVE_PATH]
                    [--FEATURE_EXTRACTOR_SAVE_PATH FEATURE_EXTRACTOR_SAVE_PATH]
test_args.py: error: unrecognized arguments: --SESSION 1 --TRAINING 1
